1. Docling
    - EasyOCR
    - TesseractOCR
    - Qari2B 

In [1]:
import json
import os
from evaluate import load
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions, AcceleratorOptions, AcceleratorDevice
from docling.models.tesseract_ocr_model import TesseractOcrOptions
from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.models.tesseract_ocr_model import TesseractOcrOptions
import re
import sys
import logging
# Setup logging
logging.basicConfig(level=logging.INFO)
_log = logging.getLogger(__name__)

In [2]:
# @lru_cache(maxsize=128)
def convert_easyocr(source, start=1, end=None):
    """Convert using EasyOCR as the default OCR engine."""
    pipeline_options = PdfPipelineOptions()
    pipeline_options.do_ocr = True
    pipeline_options.do_table_structure = True
    pipeline_options.table_structure_options.do_cell_matching = True
    pipeline_options.ocr_options.lang = ["ar", "en"]  # Default to EasyOCR per docs
    pipeline_options.accelerator_options = AcceleratorOptions(
        num_threads=6, device=AcceleratorDevice.AUTO
    )
    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )
    if end is None:
        end = sys.maxsize
    page_range = (start, end)
    _log.info("Running EasyOCR conversion...")
    return doc_converter.convert(source, page_range=page_range)
# @lru_cache(maxsize=128)
def convert_tesseract(source, start=1, end=None):
    """Convert using Tesseract OCR explicitly."""
    pipeline_options = PdfPipelineOptions()
    pipeline_options.do_ocr = True
    pipeline_options.do_table_structure = True
    pipeline_options.table_structure_options.do_cell_matching = True
    pipeline_options.ocr_options = TesseractOcrOptions(lang=["ara"])
    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )
    if end is None:
        end = sys.maxsize
    page_range = (start, end)
    _log.info("Running Tesseract conversion...")
    return doc_converter.convert(source, page_range=page_range)


In [3]:
source = "../cold_war_data/cold_war.pdf"

result_tessract = convert_tesseract(source,start=9,end=15)


INFO:__main__:Running Tesseract conversion...
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'


/home/ai/miniconda3/envs/nlp/lib/python3.11/site-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.pipeline.base_pipeline:Processing document cold_war.pdf
INFO:docling.document_converter:Finished converting document cold_war.pdf in 63.28 sec.


In [4]:

result_easy = convert_easyocr(source,start=9,end=15)


INFO:__main__:Running EasyOCR conversion...
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.pipeline.base_pipeline:Processing document cold_war.pdf
INFO:docling.document_converter:Finished converting document cold_war.pdf in 5.33 sec.


## Iterate over pages

In [5]:
import re
DIACRITICS = re.compile(r'[\u064b-\u065f]')


def clean_text(text):
    cleaned = re.sub(r'(?<![\u0600-\u06FF])' + DIACRITICS.pattern + r'(?![\u0600-\u06FF])', '', text)
    return cleaned



In [6]:
page_texts = {}
for item in result_tessract.document.texts:  # Iterate over the 'texts' list
    page_no = item.prov[0].page_no
    text = clean_text(item.text)
    if not text:
        continue
    
    if page_no not in page_texts:
        page_texts[page_no] = []
    page_texts[page_no].append(item.text)


for page_num in range(9, 15):
    print(f"Page {page_num} Text:")
    if page_num in page_texts and page_texts[page_num]:
        combined_text = " ".join(page_texts[page_num])
        print(f"{combined_text}")
    else:
        print("(No text found)")


    print('-' * 50)

Page 9 Text:
ﻣﻘﺪﻣﺔ اﻷﻋﲈل اﻟﻜﺎﻣﻠﺔ ﻟﻠﻜﺎﺗﺐ واﳌﱰﺟﻢ ﻃﻠﻌﺖ اﻟﺸﺎﻳﺐ ﻛﺘﺎﺑﺔ ﻣﻘﺪﻣﺔ ﻷﻋﻤﺎل واﻟﺪي اﻟﻜﺎﻣﻠﺔ وإﺳﻬﺎﻣﺎﺗﻪ ﰲ « ﻫﻨﺪاوي » ﺣﻴﻨﻤﺎ ﻃﻠﺒَﺖ ﻣﻨﻲ دار اﻟﻨﴩ ﻣﺠﺎل اﻟﱰﺟﻤﺔ، ﻗﻔﺰَت إﱃ ذﻫﻨﻲ ﻣُﺒﺎﴍة ً ﺻﻮرﺗ ُ ﻪ ﰲ ﺟﻠﺴﺘﻪ اﻟﺪﱠءُوﺑﺔ ﺳﺎﻋﺎت ﻃﻮﻳﻠﺔ ﰲ ﻏﺮﻓﺔ ﻣﻜﺘﺒﻪ ﻣُﺤﺎﻃ ً ﺎ ﺑﻌﴩات اﻟﻜﺘﺐ والم َ ﺮاﺟﻊ واﻟﻘﻮاﻣﻴﺲ. ﱠ والم ُ ﻔﻜﺮﻳﻦ اﻟﻜ ُﺘﺎب ﻧ َ ﻬ ِ ﻤ ً ﺎ وﻣ ُ ﺘﺎﺑ ِ ﻌ ً ﺎ دﻗﻴﻘ ً ﺎ ﻟﻜﻞ اﻹﺻﺪارات اﻟﺤﺪﻳﺜﺔ لمﻌﻈﻢ ً ﺎ ﻛﺎن أﺑﻲ ﻗﺎرﺋ واﻷ ُ دﺑﺎء اﻟﻌﺮب واﻷﺟﺎﻧﺐ، ﻟﻜﻦ أﻣﺘﻊَ ﻟﺤﻈﺎﺗﻪ ﻋﲆ اﻹﻃﻼق ﺗﻠﻚ اﻟﺘﻲ ﻳَﻘﻀﻴﻬﺎ ﰲ ﺗﺮﺟﻤﺔ ِ ﻋﻤﻞ وﻧﻘﻠﻪ ﻣﻦ ﻟ ُ ﻐﺘﻪ اﻷم إﱃ اﻟﻠﻐﺔ اﻟﻌﺮﺑﻴﺔ. ﻳﻨﺸﻐﻞ أﻳﺎﻣ ً ﺎ ﻟﻠﻌﺜﻮر ﻋﲆ اﻟﺘﻌﺒير المﻨﺎﺳﺐ أو اﻟﻜﻠﻤﺔ اﻟﺪﻗﻴﻘﺔ أو المﻘﺎﺑﻞ اﻟﻠﻐﻮي اﻟﺼﺤﻴﺢ اﻟﺬي ﻳَﻨﻘﻞ روح اﻟﻨﺺ وﻟﻴﺲ المﻌﻨﻰ اﻟﺤﺮﰲ؛ ﻣﻬﻤﺔ ﻟﻢ ﺗﻜﻦ ﻗ َ ﻂ ﺳﻬﻠﺔ، ﺧﺎﺻﺔ ﻋﻨﺪ ﺗﺮﺟﻤﺔ اﻟﺸﻌﺮ أو اﻷدب اﻟﻠﺬ َ ﻳﻦ ﻛﺎن ﻣ ُ ﻮﻟ َ ﻌ ً ﺎ ﺑﻬﻤﺎ ﰲ اﻷﺳﺎس. ﱡ اﺣﱰف أﺑﻲ اﻟﱰﺟﻤﺔ ﻣﻦ وﺣﻲ اﺣﱰاﻓﻪ اﻟﻘﺮاءة واﻟﻨﻘﺪ ﰲ زﻣﻦ ﻟﻢ ﺗﻜﻦ ﻓﻴﻪ ﻣﺼﺎدر اﻟﺒﺤﺚ ﻋﱪ اﻹﻧﱰﻧﺖ ﻣﺘﻮاﻓﺮة ً ﻛﻤﺎ ﻫﻲ اﻵن؛ ﺑﻜﺒﺴﺔ زرﱟ ﺗﺴﺘﻄﻴﻊ اﻟﻌﺜﻮرَ ﻋﲆ ﻣﺼﻄﻠﺤﺎت أو ﻣﻌﻠﻮﻣﺎت أو ﺗﻔﺎﺻﻴﻞ ﻋﻦ ﺣﺪَث ﺗﺎرﻳﺨﻲ. أو أﻳﺎﻣًﺎ ﻟﻠﻌﺜﻮر ﻋﲆ ﻣُﺮادف ﻟﻪ ﻣﺪﻟﻮل ﺛﻘﺎﰲ الم َ ﺮاﺟﻊ واﻟﻜﺘﺐ ﻛﺎن ﻋﻠﻴﻪ اﻟﺒﺤﺚ ﰲ ﻣﻌﻠﻮﻣﺎت ﻋﻦ ﺣﺪث ﺗﺎرﻳﺨﻲ 

In [7]:
page_texts = {}
for item in result_easy.document.texts:  # Iterate over the 'texts' list
    page_no = item.prov[0].page_no
    text = clean_text(item.text)
    if not text:
        continue
    
    if page_no not in page_texts:
        page_texts[page_no] = []
    page_texts[page_no].append(item.text)


for page_num in range(9, 15):
    print(f"Page {page_num} Text:")
    if page_num in page_texts and page_texts[page_num]:
        combined_text = " ".join(page_texts[page_num])
        print(f"{combined_text}")
    else:
        print("(No text found)")


    print('-' * 50)

Page 9 Text:
ﻣﻘﺪﻣﺔ اﻷﻋﲈل اﻟﻜﺎﻣﻠﺔ ﻟﻠﻜﺎﺗﺐ واﳌﱰﺟﻢ ﻃﻠﻌﺖ اﻟﺸﺎﻳﺐ ﻛﺘﺎﺑﺔ ﻣﻘﺪﻣﺔ ﻷﻋﻤﺎل واﻟﺪي اﻟﻜﺎﻣﻠﺔ وإﺳﻬﺎﻣﺎﺗﻪ ﰲ « ﻫﻨﺪاوي » ﺣﻴﻨﻤﺎ ﻃﻠﺒَﺖ ﻣﻨﻲ دار اﻟﻨﴩ ﻣﺠﺎل اﻟﱰﺟﻤﺔ، ﻗﻔﺰَت إﱃ ذﻫﻨﻲ ﻣُﺒﺎﴍة ً ﺻﻮرﺗ ُ ﻪ ﰲ ﺟﻠﺴﺘﻪ اﻟﺪﱠءُوﺑﺔ ﺳﺎﻋﺎت ﻃﻮﻳﻠﺔ ﰲ ﻏﺮﻓﺔ ﻣﻜﺘﺒﻪ ﻣُﺤﺎﻃ ً ﺎ ﺑﻌﴩات اﻟﻜﺘﺐ والم َ ﺮاﺟﻊ واﻟﻘﻮاﻣﻴﺲ. ﱠ والم ُ ﻔﻜﺮﻳﻦ اﻟﻜ ُﺘﺎب ﻧ َ ﻬ ِ ﻤ ً ﺎ وﻣ ُ ﺘﺎﺑ ِ ﻌ ً ﺎ دﻗﻴﻘ ً ﺎ ﻟﻜﻞ اﻹﺻﺪارات اﻟﺤﺪﻳﺜﺔ لمﻌﻈﻢ ً ﺎ ﻛﺎن أﺑﻲ ﻗﺎرﺋ واﻷ ُ دﺑﺎء اﻟﻌﺮب واﻷﺟﺎﻧﺐ، ﻟﻜﻦ أﻣﺘﻊَ ﻟﺤﻈﺎﺗﻪ ﻋﲆ اﻹﻃﻼق ﺗﻠﻚ اﻟﺘﻲ ﻳَﻘﻀﻴﻬﺎ ﰲ ﺗﺮﺟﻤﺔ ِ ﻋﻤﻞ وﻧﻘﻠﻪ ﻣﻦ ﻟ ُ ﻐﺘﻪ اﻷم إﱃ اﻟﻠﻐﺔ اﻟﻌﺮﺑﻴﺔ. ﻳﻨﺸﻐﻞ أﻳﺎﻣ ً ﺎ ﻟﻠﻌﺜﻮر ﻋﲆ اﻟﺘﻌﺒير المﻨﺎﺳﺐ أو اﻟﻜﻠﻤﺔ اﻟﺪﻗﻴﻘﺔ أو المﻘﺎﺑﻞ اﻟﻠﻐﻮي اﻟﺼﺤﻴﺢ اﻟﺬي ﻳَﻨﻘﻞ روح اﻟﻨﺺ وﻟﻴﺲ المﻌﻨﻰ اﻟﺤﺮﰲ؛ ﻣﻬﻤﺔ ﻟﻢ ﺗﻜﻦ ﻗ َ ﻂ ﺳﻬﻠﺔ، ﺧﺎﺻﺔ ﻋﻨﺪ ﺗﺮﺟﻤﺔ اﻟﺸﻌﺮ أو اﻷدب اﻟﻠﺬ َ ﻳﻦ ﻛﺎن ﻣ ُ ﻮﻟ َ ﻌ ً ﺎ ﺑﻬﻤﺎ ﰲ اﻷﺳﺎس. ﱡ اﺣﱰف أﺑﻲ اﻟﱰﺟﻤﺔ ﻣﻦ وﺣﻲ اﺣﱰاﻓﻪ اﻟﻘﺮاءة واﻟﻨﻘﺪ ﰲ زﻣﻦ ﻟﻢ ﺗﻜﻦ ﻓﻴﻪ ﻣﺼﺎدر اﻟﺒﺤﺚ ﻋﱪ اﻹﻧﱰﻧﺖ ﻣﺘﻮاﻓﺮة ً ﻛﻤﺎ ﻫﻲ اﻵن؛ ﺑﻜﺒﺴﺔ زرﱟ ﺗﺴﺘﻄﻴﻊ اﻟﻌﺜﻮرَ ﻋﲆ ﻣﺼﻄﻠﺤﺎت أو ﻣﻌﻠﻮﻣﺎت أو ﺗﻔﺎﺻﻴﻞ ﻋﻦ ﺣﺪَث ﺗﺎرﻳﺨﻲ. أو أﻳﺎﻣًﺎ ﻟﻠﻌﺜﻮر ﻋﲆ ﻣُﺮادف ﻟﻪ ﻣﺪﻟﻮل ﺛﻘﺎﰲ الم َ ﺮاﺟﻊ واﻟﻜﺘﺐ ﻛﺎن ﻋﻠﻴﻪ اﻟﺒﺤﺚ ﰲ ﻣﻌﻠﻮﻣﺎت ﻋﻦ ﺣﺪث ﺗﺎرﻳﺨﻲ 

In [8]:
def extract_docling_text(result):
    page_texts = {}
    for item in result.document.texts:
        page_no = item.prov[0].page_no
        text = clean_text(item.text)
        if not text:
            continue

        if page_no not in page_texts:
            page_texts[page_no] = []
        page_texts[page_no].append(item.text)

    hypothesis = {}
    for page_num in page_texts:
        hypothesis[page_num] = " ".join(page_texts[page_num])
    return hypothesis

In [9]:
extract_docling_text(result_easy)

{9: 'ﻣﻘﺪﻣﺔ اﻷﻋﲈل اﻟﻜﺎﻣﻠﺔ ﻟﻠﻜﺎﺗﺐ واﳌﱰﺟﻢ ﻃﻠﻌﺖ اﻟﺸﺎﻳﺐ ﻛﺘﺎﺑﺔ ﻣﻘﺪﻣﺔ ﻷﻋﻤﺎل واﻟﺪي اﻟﻜﺎﻣﻠﺔ وإﺳﻬﺎﻣﺎﺗﻪ ﰲ « ﻫﻨﺪاوي » ﺣﻴﻨﻤﺎ ﻃﻠﺒَﺖ ﻣﻨﻲ دار اﻟﻨﴩ ﻣﺠﺎل اﻟﱰﺟﻤﺔ، ﻗﻔﺰَت إﱃ ذﻫﻨﻲ ﻣُﺒﺎﴍة ً ﺻﻮرﺗ ُ ﻪ ﰲ ﺟﻠﺴﺘﻪ اﻟﺪﱠءُوﺑﺔ ﺳﺎﻋﺎت ﻃﻮﻳﻠﺔ ﰲ ﻏﺮﻓﺔ ﻣﻜﺘﺒﻪ ﻣُﺤﺎﻃ ً ﺎ ﺑﻌﴩات اﻟﻜﺘﺐ والم َ ﺮاﺟﻊ واﻟﻘﻮاﻣﻴﺲ. ﱠ والم ُ ﻔﻜﺮﻳﻦ اﻟﻜ ُﺘﺎب ﻧ َ ﻬ ِ ﻤ ً ﺎ وﻣ ُ ﺘﺎﺑ ِ ﻌ ً ﺎ دﻗﻴﻘ ً ﺎ ﻟﻜﻞ اﻹﺻﺪارات اﻟﺤﺪﻳﺜﺔ لمﻌﻈﻢ ً ﺎ ﻛﺎن أﺑﻲ ﻗﺎرﺋ واﻷ ُ دﺑﺎء اﻟﻌﺮب واﻷﺟﺎﻧﺐ، ﻟﻜﻦ أﻣﺘﻊَ ﻟﺤﻈﺎﺗﻪ ﻋﲆ اﻹﻃﻼق ﺗﻠﻚ اﻟﺘﻲ ﻳَﻘﻀﻴﻬﺎ ﰲ ﺗﺮﺟﻤﺔ ِ ﻋﻤﻞ وﻧﻘﻠﻪ ﻣﻦ ﻟ ُ ﻐﺘﻪ اﻷم إﱃ اﻟﻠﻐﺔ اﻟﻌﺮﺑﻴﺔ. ﻳﻨﺸﻐﻞ أﻳﺎﻣ ً ﺎ ﻟﻠﻌﺜﻮر ﻋﲆ اﻟﺘﻌﺒير المﻨﺎﺳﺐ أو اﻟﻜﻠﻤﺔ اﻟﺪﻗﻴﻘﺔ أو المﻘﺎﺑﻞ اﻟﻠﻐﻮي اﻟﺼﺤﻴﺢ اﻟﺬي ﻳَﻨﻘﻞ روح اﻟﻨﺺ وﻟﻴﺲ المﻌﻨﻰ اﻟﺤﺮﰲ؛ ﻣﻬﻤﺔ ﻟﻢ ﺗﻜﻦ ﻗ َ ﻂ ﺳﻬﻠﺔ، ﺧﺎﺻﺔ ﻋﻨﺪ ﺗﺮﺟﻤﺔ اﻟﺸﻌﺮ أو اﻷدب اﻟﻠﺬ َ ﻳﻦ ﻛﺎن ﻣ ُ ﻮﻟ َ ﻌ ً ﺎ ﺑﻬﻤﺎ ﰲ اﻷﺳﺎس. ﱡ اﺣﱰف أﺑﻲ اﻟﱰﺟﻤﺔ ﻣﻦ وﺣﻲ اﺣﱰاﻓﻪ اﻟﻘﺮاءة واﻟﻨﻘﺪ ﰲ زﻣﻦ ﻟﻢ ﺗﻜﻦ ﻓﻴﻪ ﻣﺼﺎدر اﻟﺒﺤﺚ ﻋﱪ اﻹﻧﱰﻧﺖ ﻣﺘﻮاﻓﺮة ً ﻛﻤﺎ ﻫﻲ اﻵن؛ ﺑﻜﺒﺴﺔ زرﱟ ﺗﺴﺘﻄﻴﻊ اﻟﻌﺜﻮرَ ﻋﲆ ﻣﺼﻄﻠﺤﺎت أو ﻣﻌﻠﻮﻣﺎت أو ﺗﻔﺎﺻﻴﻞ ﻋﻦ ﺣﺪَث ﺗﺎرﻳﺨﻲ. أو أﻳﺎﻣًﺎ ﻟﻠﻌﺜﻮر ﻋﲆ ﻣُﺮادف ﻟﻪ ﻣﺪﻟﻮل ﺛﻘﺎﰲ الم َ ﺮاﺟﻊ واﻟﻜﺘﺐ ﻛﺎن ﻋﻠﻴﻪ اﻟﺒﺤﺚ ﰲ ﻣﻌﻠﻮﻣﺎت ﻋﻦ ﺣﺪث ﺗﺎرﻳﺨﻲ ور َدَ ﰲ

## Benchmarking

In [10]:
from evaluate import  load
import os 
import json

In [11]:
def load_ground_truth(directory, start_page, end_page):
    """Load ground truth text from JSON files."""
    ground_truth = {}
    for page_num in range(start_page, end_page + 1):
        file_path = os.path.join(directory, f"page_{page_num}.json")
        if os.path.exists(file_path):
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                ground_truth[page_num] = clean_text(" ".join(data["content"]))
    return ground_truth

In [12]:
def compute_metrics(ground_truth, hypothesis, label=""):
    """Compute WER and CER using Hugging Face evaluate library."""
    wer_metric = load("wer")
    cer_metric = load("cer")
    results = {}
    total_wer, total_cer, page_count = 0, 0, 0

    for page_num in ground_truth.keys():
        if page_num in hypothesis:
            ref_text = ground_truth[page_num]
            hyp_text = hypothesis[page_num]
            wer = wer_metric.compute(predictions=[hyp_text], references=[ref_text])
            cer = cer_metric.compute(predictions=[hyp_text], references=[ref_text])
            results[page_num] = {"wer": wer, "cer": cer}

            print(f"{label} Page {page_num}:")
            print(f"  Ground Truth: {ref_text[:100]}...")
            print(f"  Hypothesis:   {hyp_text[:100]}...")
            print(f"  WER: {wer:.4f}")
            print(f"  CER: {cer:.4f}")
            print('-' * 50)

            total_wer += wer
            total_cer += cer
            page_count += 1

    if page_count > 0:
        avg_wer = total_wer / page_count
        avg_cer = total_cer / page_count
        print(f"{label} Average WER: {avg_wer:.4f}")
        print(f"{label} Average CER: {avg_cer:.4f}")
    return results



In [13]:
source = "../cold_war_data/cold_war.pdf"
json_directory = "../cold_war_data"
start_page = 9
end_page = 15

ground_truth = load_ground_truth(json_directory, start_page, end_page)
if not ground_truth:
    print("No ground truth data found.")


In [14]:

result_tessract = convert_tesseract(source, start=start_page, end=end_page)
hypothesis_tess = extract_docling_text(result_tessract)
if not hypothesis_tess:
    print("No Tesseract hypothesis extracted.")
else:
    print("Tesseract Results:")
    metrics_tess = compute_metrics(ground_truth, hypothesis_tess, label="Tesseract")



INFO:__main__:Running Tesseract conversion...
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'


INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.pipeline.base_pipeline:Processing document cold_war.pdf
INFO:docling.document_converter:Finished converting document cold_war.pdf in 2.47 sec.


Tesseract Results:
Tesseract Page 9:
  Ground Truth: مقدمة الأعمال الكاملة للكاتب والمترجم طلعت الشايب حينما طلبت مني دار النشر «هنداوي» كتابة مقدمة لأعم...
  Hypothesis:   ﻣﻘﺪﻣﺔ اﻷﻋﲈل اﻟﻜﺎﻣﻠﺔ ﻟﻠﻜﺎﺗﺐ واﳌﱰﺟﻢ ﻃﻠﻌﺖ اﻟﺸﺎﻳﺐ ﻛﺘﺎﺑﺔ ﻣﻘﺪﻣﺔ ﻷﻋﻤﺎل واﻟﺪي اﻟﻜﺎﻣﻠﺔ وإﺳﻬﺎﻣﺎﺗﻪ ﰲ « ﻫﻨﺪاوي »...
  WER: 1.2174
  CER: 0.7143
--------------------------------------------------
Tesseract Page 10:
  Ground Truth: أدب الحرب الباردة اتَّبِعي قلبك»، و«أصوات الضمير»، و«بقايا اليوم»، و«هوس العمق»، و«الخوف من المرايا»...
  Hypothesis:   أدب اﻟﺤﺮب اﻟﺒﺎردة ، « اﻟﺨﻮف ﻣﻦ المﺮاﻳﺎ » ، و « ﺑﻘﺎﻳﺎ اﻟﻴﻮم ، و ﻫﻮس اﻟﻌﻤﻖ » « » ، و « اﺗ ﱠ ﺒ ِ ﻌﻲ ﻗﻠﺒ...
  WER: 1.3491
  CER: 0.7579
--------------------------------------------------
Tesseract Page 11:
  Ground Truth: مقدمة المحرر بين الخطاب والرد: (أفكار تمهيدية عن كتابة الحرب الباردة) أندرو هاموند الحرب الباردة» مص...
  Hypothesis:   ﱢ ر ﻣﻘﺪ ﱢ ﻣﺔ اﳌﺤﺮ ﺑين اﻟﺨﻄﺎب واﻟﺮد: )أﻓﻜﺎر ﺗﻤﻬﻴﺪﻳﺔ ﻋﻦ ﻛﺘﺎﺑﺔ اﻟﺤﺮب اﻟﺒﺎردة( أﻧﺪرو ﻫﺎﻣﻮﻧﺪ ﻗﺎرات ﻣﺼﻄﻠﺢ ...
  WER: 1.0489
  CER: 0.6569
-------------------

In [15]:

result_easy = convert_easyocr(source, start=start_page, end=end_page)
hypothesis_easy = extract_docling_text(result_easy)
if not hypothesis_easy:
    print("No EasyOCR hypothesis extracted.")
else:
    print("\nEasyOCR Results:")
    metrics_easy = compute_metrics(ground_truth, hypothesis_easy, label="EasyOCR")



INFO:__main__:Running EasyOCR conversion...
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cuda:0'
INFO:docling.pipeline.base_pipeline:Processing document cold_war.pdf
INFO:docling.document_converter:Finished converting document cold_war.pdf in 5.26 sec.



EasyOCR Results:
EasyOCR Page 9:
  Ground Truth: مقدمة الأعمال الكاملة للكاتب والمترجم طلعت الشايب حينما طلبت مني دار النشر «هنداوي» كتابة مقدمة لأعم...
  Hypothesis:   ﻣﻘﺪﻣﺔ اﻷﻋﲈل اﻟﻜﺎﻣﻠﺔ ﻟﻠﻜﺎﺗﺐ واﳌﱰﺟﻢ ﻃﻠﻌﺖ اﻟﺸﺎﻳﺐ ﻛﺘﺎﺑﺔ ﻣﻘﺪﻣﺔ ﻷﻋﻤﺎل واﻟﺪي اﻟﻜﺎﻣﻠﺔ وإﺳﻬﺎﻣﺎﺗﻪ ﰲ « ﻫﻨﺪاوي »...
  WER: 1.2174
  CER: 0.7143
--------------------------------------------------
EasyOCR Page 10:
  Ground Truth: أدب الحرب الباردة اتَّبِعي قلبك»، و«أصوات الضمير»، و«بقايا اليوم»، و«هوس العمق»، و«الخوف من المرايا»...
  Hypothesis:   أدب اﻟﺤﺮب اﻟﺒﺎردة ، « اﻟﺨﻮف ﻣﻦ المﺮاﻳﺎ » ، و « ﺑﻘﺎﻳﺎ اﻟﻴﻮم ، و ﻫﻮس اﻟﻌﻤﻖ » « » ، و « اﺗ ﱠ ﺒ ِ ﻌﻲ ﻗﻠﺒ...
  WER: 1.3491
  CER: 0.7579
--------------------------------------------------
EasyOCR Page 11:
  Ground Truth: مقدمة المحرر بين الخطاب والرد: (أفكار تمهيدية عن كتابة الحرب الباردة) أندرو هاموند الحرب الباردة» مص...
  Hypothesis:   ﱢ ر ﻣﻘﺪ ﱢ ﻣﺔ اﳌﺤﺮ ﺑين اﻟﺨﻄﺎب واﻟﺮد: )أﻓﻜﺎر ﺗﻤﻬﻴﺪﻳﺔ ﻋﻦ ﻛﺘﺎﺑﺔ اﻟﺤﺮب اﻟﺒﺎردة( أﻧﺪرو ﻫﺎﻣﻮﻧﺪ ﻗﺎرات ﻣﺼﻄﻠﺢ ...
  WER: 1.0489
  CER: 0.6569
--------------------------

In [16]:

sample_page = 9
if sample_page in hypothesis_tess and sample_page in hypothesis_easy:
    print("\nDebug Comparison for Page 9:")
    print(f"Tesseract: {hypothesis_tess[sample_page][:200]}...")
    print(f"EasyOCR:   {hypothesis_easy[sample_page][:200]}...")


Debug Comparison for Page 9:
Tesseract: ﻣﻘﺪﻣﺔ اﻷﻋﲈل اﻟﻜﺎﻣﻠﺔ ﻟﻠﻜﺎﺗﺐ واﳌﱰﺟﻢ ﻃﻠﻌﺖ اﻟﺸﺎﻳﺐ ﻛﺘﺎﺑﺔ ﻣﻘﺪﻣﺔ ﻷﻋﻤﺎل واﻟﺪي اﻟﻜﺎﻣﻠﺔ وإﺳﻬﺎﻣﺎﺗﻪ ﰲ « ﻫﻨﺪاوي » ﺣﻴﻨﻤﺎ ﻃﻠﺒَﺖ ﻣﻨﻲ دار اﻟﻨﴩ ﻣﺠﺎل اﻟﱰﺟﻤﺔ، ﻗﻔﺰَت إﱃ ذﻫﻨﻲ ﻣُﺒﺎﴍة ً ﺻﻮرﺗ ُ ﻪ ﰲ ﺟﻠﺴﺘﻪ اﻟﺪﱠءُوﺑﺔ ﺳﺎﻋﺎت ﻃﻮﻳﻠﺔ...
EasyOCR:   ﻣﻘﺪﻣﺔ اﻷﻋﲈل اﻟﻜﺎﻣﻠﺔ ﻟﻠﻜﺎﺗﺐ واﳌﱰﺟﻢ ﻃﻠﻌﺖ اﻟﺸﺎﻳﺐ ﻛﺘﺎﺑﺔ ﻣﻘﺪﻣﺔ ﻷﻋﻤﺎل واﻟﺪي اﻟﻜﺎﻣﻠﺔ وإﺳﻬﺎﻣﺎﺗﻪ ﰲ « ﻫﻨﺪاوي » ﺣﻴﻨﻤﺎ ﻃﻠﺒَﺖ ﻣﻨﻲ دار اﻟﻨﴩ ﻣﺠﺎل اﻟﱰﺟﻤﺔ، ﻗﻔﺰَت إﱃ ذﻫﻨﻲ ﻣُﺒﺎﴍة ً ﺻﻮرﺗ ُ ﻪ ﰲ ﺟﻠﺴﺘﻪ اﻟﺪﱠءُوﺑﺔ ﺳﺎﻋﺎت ﻃﻮﻳﻠﺔ...


## VLlm

In [17]:
import os 
from pdf2image import  convert_from_path
from transformers import  Qwen2VLForConditionalGeneration,AutoProcessor
from qwen_vl_utils import  process_vision_info
import torch 
from typing import  Dict

INFO:qwen_vl_utils.vision_process:set VIDEO_TOTAL_PIXELS: 90316800


In [ ]:
def convert_pdf_with_qwen(pdf_path: str, start_page: int = 1, end_page: int = None) -> Dict[int, str]:
    """
    Extract text from PDF pages using Qwen2.5-VL-3B-Instruct.
    
    Args:
        pdf_path (str): Path to the PDF file.
        start_page (int): Starting page number (1-based indexing).
        end_page (int, optional): Ending page number (1-based indexing). If None, processes all pages from start_page.
    
    Returns:
        Dict[int, str]: Dictionary mapping page numbers to extracted text.
    """
    # Adjust for 0-based indexing used by pdf2image
    start_idx = start_page - 1
    end_idx = end_page if end_page is not None else None

    # Convert PDF pages to images
    _log.info(f"Converting PDF {pdf_path} to images for pages {start_page} to {end_page or 'end'}...")
    try:
        images = convert_from_path(pdf_path, first_page=start_page, last_page=end_page)
    except Exception as e:
        _log.error(f"Failed to convert PDF to images: {e}")
        return {}

    if not images:
        _log.warning("No images extracted from PDF.")
        return {}

    # Load Qwen2.5-VL model and processor
    model_name = "NAMAA-Space/Qari-OCR-0.2.2-Arabic-2B-Instruct"
    _log.info(f"Loading model {model_name}...")
    try:
        model = Qwen2VLForConditionalGeneration.from_pretrained(
            model_name, torch_dtype="auto", device_map="cuda:0"
        )
        processor = AutoProcessor.from_pretrained(model_name)
    except Exception as e:
        _log.error(f"Failed to load Qwen2.5-VL model: {e}")
        return {}

    # Move model to GPU if available
    if torch.cuda.is_available():
        model = model.to("cuda")
        _log.info("Model loaded on CUDA.")
    else:
        _log.warning("CUDA not available, running on CPU. This may be slow.")

    # Prepare OCR prompt
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": None},  # Placeholder for image
                {"type": "text", "text": "Extract all text from this image."},
            ],
        }
    ]

    # Process each page
    extracted_texts = {}
    for i, image in enumerate(images, start=start_idx):
        page_num = i + 1  # Convert back to 1-based indexing
        _log.info(f"Processing page {page_num}...")

        # Update message with current image
        # messages[0]["content"][0]["image"] = image.resize((850,850))  # Pass PIL image directly
        messages[0]["content"][0]["image"] = image

        # Prepare inputs for inference
        try:
            text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            image_inputs, video_inputs = process_vision_info(messages)
            inputs = processor(
                text=[text],
                images=image_inputs,
                videos=video_inputs,
                padding=True,
                return_tensors="pt",
            )
            if torch.cuda.is_available():
                inputs = inputs.to("cuda:0")

            # Generate output
            generated_ids = model.generate(**inputs, max_new_tokens=4096)
            generated_ids_trimmed = [
                out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            output_text = processor.batch_decode(
                generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
            )[0]
            extracted_texts[page_num] = output_text.strip()
        except Exception as e:
            _log.error(f"Error processing page {page_num}: {e}")
            extracted_texts[page_num] = f"Error: {str(e)}"

    return extracted_texts



In [19]:

result = convert_pdf_with_qwen(source, start_page=9, end_page=15)

# Print results
for page_num, text in result.items():
    print(f"Page {page_num} Text:")
    print(f"{text}")
    print('-' * 50)

INFO:__main__:Converting PDF ../cold_war_data/cold_war.pdf to images for pages 9 to 15...
INFO:__main__:Loading model NAMAA-Space/Qari-OCR-0.2.2-Arabic-2B-Instruct...
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
INFO:__main__:Model loaded on CUDA.
INFO:__main__:Processing page 9...
INFO:__main__:Processing page 10...
INFO:__main__:Processing page 11...
INFO:__main__:Processing page 12...
INFO:__main__:Processing page 13...
INFO:__main__:Processing page 14...
INFO:__main__:Processing page 15...


Page 9 Text:
مقدمة الأعمال الكاملة للكاتب والمترجم طلعت الشايب حينما طلبَت مني دار النشر «هنداوي» كتابةً مقدمةٍ لأعمال والدي الكاملة وإسهاماته في مجال الترجمة، قفزَت إلى ذهني مُباشرةً صورتْه في جلسته الدَّءْوبة ساعاتٍ طويلة في غرفة مكتبه مُحاطًا بعشرات الكتب والمَراجع والقواميس. كان أبي قارثًا نَهمًا ومُتابِعًا دقيقًا لكل الإصدارات الحديثة لعظم الكُتاب والْفكرين والأُدباء العرب والأجانب، لكنَّ أمتعَ لحظاته على الإطلاق تلك التي يَقضيها في ترجمة عملٍ ونقله من لْغته الأم إلى اللغة العربية. ينشغل أيَامًا للعثور على التعبير المناسب أو الكلمة الدقيقة أو اللقابل اللغوي الصحيح الذي يَنقل روحَ النص وليس المعنى الحرفِ؛ مهمةٌ لم تكن قَطُّ سهلة، خاصةً عند ترجمة الشعر أو الأدب اللذَين كان مُولَعًا بهما في الأساس. احترف أبي الترجمةَ من وحي احترافه القراءةَ والنقد في زمنٍ لم تكن فيه مصادرُ البحث عبر الإنترنت متوافرةً كما هي الآن؛ بكبسِةٍ زرٌ تستطيع العثورَ على مصطلحاتٍ أو معلوماتٍ أو تفاصيل عن حدَثٍ تاريخي. كان عليه البحث في المَراجع والكتب أيَامًا للعثور على مُرادفٍ له مدلولٌ ثقافيٍ أو معلومات عن حد

In [21]:
print("\nQwen2.5-VL Results:")
metrics_qwen = compute_metrics(ground_truth, result, label="Qari model")


Qwen2.5-VL Results:
Qari model Page 9:
  Ground Truth: مقدمة الأعمال الكاملة للكاتب والمترجم طلعت الشايب حينما طلبت مني دار النشر «هنداوي» كتابة مقدمة لأعم...
  Hypothesis:   مقدمة الأعمال الكاملة للكاتب والمترجم طلعت الشايب حينما طلبَت مني دار النشر «هنداوي» كتابةً مقدمةٍ ل...
  WER: 0.2899
  CER: 0.0644
--------------------------------------------------
Qari model Page 10:
  Ground Truth: أدب الحرب الباردة اتَّبِعي قلبك»، و«أصوات الضمير»، و«بقايا اليوم»، و«هوس العمق»، و«الخوف من المرايا»...
  Hypothesis:   أدب الحرب الباردة «اتِّبعي قلبك»، و«أصوات الضمير»، و«بقايا اليوم»، و«هوس العمق»، و«الخوف من المرايا»...
  WER: 0.2109
  CER: 0.0494
--------------------------------------------------
Qari model Page 11:
  Ground Truth: مقدمة المحرر بين الخطاب والرد: (أفكار تمهيدية عن كتابة الحرب الباردة) أندرو هاموند الحرب الباردة» مص...
  Hypothesis:   مقدّمة الحرّر بين الخطاب والرد: (أفكار تمهيدية عن كتابة الحرب الباردة) أندرو هاموند «الحرب الباردة» ...
  WER: 0.1467
  CER: 0.0285
--------------